In [20]:
import pandas as pd
import numpy as np

In [16]:
col_type_dict = {
    'timestamp': 'datetime64[ns]',
    'measuring_station': 'string',
    'soil_moisture': 'float64',
    'soil_temperature': 'float64',
    'soil_salinity': 'float64',
    'soil_ph': 'float64',
    'battery_voltage': 'float64',
    'irrigated': 'float64',
    'reward': 'float64'
}

In [ ]:
exc_file_path = 'data/EARTH2301.xlsx'
csv_file_path = 'data/cleaned/EARTH2301_output.csv'

In [19]:
print(f"Reading excel at path: '{exc_file_path}'")
pd_df = pd.read_excel(exc_file_path)

for col in pd_df.columns:
    pd_df[col] = pd_df[col].astype(col_type_dict[col])

print(pd_df.dtypes)


Reading excel at path: 'data/EARTH2301.xlsx'
timestamp            datetime64[ns]
measuring_station    string[python]
soil_moisture               float64
soil_temperature            float64
soil_salinity               float64
soil_ph                     float64
battery_voltage             float64
irrigated                   float64
reward                      float64
dtype: object


In [21]:
# pd_df['irrigated'] = pd_df['soil_moisture'].apply(lambda x: 5 * round((30 - x) / 5))
new_df = pd_df.copy()

values = [0, 5, 10, 15, 20, 25]
new_df['irrigated'] = np.random.choice(values, size=len(pd_df))
# pd_df['reward'] = None
# pd_df['reward'] = pd_df['reward'].astype('float64')

print(new_df.dtypes)
new_df.head(5)

timestamp            datetime64[ns]
measuring_station    string[python]
soil_moisture               float64
soil_temperature            float64
soil_salinity               float64
soil_ph                     float64
battery_voltage             float64
irrigated                     int64
reward                      float64
dtype: object


,timestamp,measuring_station,soil_moisture,soil_temperature,soil_salinity,soil_ph,battery_voltage,irrigated,reward
0,2023-01-31 23:59:00,SVDT3,25.5,24.8,20.3,6.1,12.7,15,NaN
1,2023-01-31 23:58:00,SVDT3,25.5,24.8,20.0,6.2,12.7,0,NaN
2,2023-01-31 23:57:00,SVDT3,25.5,24.8,20.0,6.3,12.7,25,NaN
3,2023-01-31 23:56:00,SVDT3,25.5,24.8,20.1,6.4,12.7,0,NaN
4,2023-01-31 23:55:00,SVDT3,25.5,24.8,20.2,6.3,12.7,15,NaN


In [25]:
def reward_simulation(row, optimal_min=30.0, optimal_max=40.0):
    """
    Calculates the reward for each row based on the soil moisture and irrigation amount.
    
    Parameters:
    row (Series): Row of the DataFrame.
    optimal_min (float): Minimum of the optimal soil moisture range.
    optimal_max (float): Maximum of the optimal soil moisture range.
    
    Returns:
    float: Reward based on the proximity of soil moisture to the optimal range.
    """
    soil_moisture = row['soil_moisture']
    irrigation = row['irrigated']
    
    # Simulate the new soil moisture after irrigation
    new_soil_moisture = soil_moisture + irrigation
    
    # Define the reward based on how close the new soil moisture is to the optimal range
    if optimal_min <= new_soil_moisture <= optimal_max:
        reward = 1.0  # High reward for being within the optimal range
    else:
        # Penalize based on the distance from the optimal range
        reward = -abs(new_soil_moisture - (optimal_min + optimal_max) / 2) / 10.0
    
    return reward

In [26]:
new_new_df = new_df.copy()
new_new_df['reward'] = new_new_df.apply(reward_simulation, axis=1)

In [29]:
new_new_df.head(1000)

,timestamp,measuring_station,soil_moisture,soil_temperature,soil_salinity,soil_ph,battery_voltage,irrigated,reward
0,2023-01-31 23:59:00,SVDT3,25.5,24.8,20.3,6.1,12.7,15,-0.55
1,2023-01-31 23:58:00,SVDT3,25.5,24.8,20.0,6.2,12.7,0,-0.95
2,2023-01-31 23:57:00,SVDT3,25.5,24.8,20.0,6.3,12.7,25,-1.55
3,2023-01-31 23:56:00,SVDT3,25.5,24.8,20.1,6.4,12.7,0,-0.95
4,2023-01-31 23:55:00,SVDT3,25.5,24.8,20.2,6.3,12.7,15,-0.55
...,...,...,...,...,...,...,...,...,...
995,2023-01-31 07:24:00,SVDT3,25.8,22.1,20.1,6.7,12.6,25,-1.58
996,2023-01-31 07:23:00,SVDT3,25.8,22.1,20.2,6.5,12.6,5,1.00
997,2023-01-31 07:22:00,SVDT3,25.8,22.1,20.3,6.7,12.6,0,-0.92
998,2023-01-31 07:21:00,SVDT3,25.9,22.1,20.3,6.4,12.6,10,1.00


In [31]:
new_new_df.to_csv(csv_file_path, index=False)